### Disclaimer

Esta aula é baseada no material disponibilizado pelo professor Jones Granatyr - IA Expert Academy.


In [1]:
import time
import random
import math
import sys

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
PATH = '/content/gdrive/MyDrive/Colab Notebooks/Inteligência Artificial/Algoritmos Genéticos/flights.txt'

#### Situação Problema

Um grupo de amigos quer ir à Roma, na Itália. Entretanto, todos estão em capitais de países diferentes. Um está em Londres, outro em Dublin, outro em Bruxelas, outro em Paris, outro em Madrid e outro em Lisboa.

Ao chegar em Roma, esse grupo de amigos irá pegar uma Van e fazer uma excursão por Roma. Depois irão pegar uma Van para voltar ao aeroporto e à suas respectivas cidades.

A ideia é tentar encontrar voos que minimizem a espera das pessoas do grupo nos aeroportos. Por exemplo, quem vem de Bruxelas pode ter que esperar o amigo de Londres, por conta dos horários dos voos. Por exemplo, considere que a última pessoa a chegar em Roma chegará às 20h. Se a pessoa que vem de Bruxelas chegar às 10h da manhã, ele terá uma espera de 10h.
Considere ainda que, na volta, a pessoa que embarcará no primeiro vôo sairá às 11h da manhã. Assim todos precisarão chegar lá pelo menos às 11h da manhã.

Além disso, queremos minimizar os custos das pessoas ao comprar as passagens.


#### Objetivos

1.   minimizar os preços das passagens;
2.   minimizar o tempo de espera das pessoas nos aeroportos, tanto na ida quanto na volta.

* Abaixo temos uma lista de tuplas com todas as origens das pessoas do grupo.

In [9]:
pessoas = [('Lisbon', 'LIS'),
           ('Madrid', 'MAD'),
           ('Paris', 'CDG'),
           ('Dublin', 'DUB'),
           ('Brussels', 'BRU'),
           ('London', 'LHR')]

* Aqui temos o destino, que é Roma (FCO):

In [10]:
destino = 'FCO' # Roma

* Precisamos carregar os dados dos voos do arquivo `flights.txt` e armezená-los em um dicionário no seguinte formtao:

In [11]:
voos = {('BRU', 'FCO'): ['15:44', '18:55', 382]}

* Exemplo de dados de voo de Bruxelas para Roma - partida às 15:44, chegada às 18h55 e custa 382 dólares:

In [12]:
voos[('BRU', 'FCO')]

['15:44', '18:55', 382]

* Como os dados do voo ficam em uma tupla, podemos acessá-los utilizando índices:

In [13]:
voos[('BRU', 'FCO')][0], voos[('BRU', 'FCO')][1], voos[('BRU', 'FCO')][2]

('15:44', '18:55', 382)

* Vamos iterar sobre os dados do arquivo `flights.txt` para carregar as dados dos voos no dicionário `voos`:

In [14]:
voos = {}
#for line in open('/content/flights.txt'):
for line in open(PATH):
    v_origem, v_destino, h_partida, h_chegada, preco_passagem = line.split(',')
    voos.setdefault((v_origem, v_destino), [])
    voos[(v_origem, v_destino)].append((h_partida, h_chegada, int(preco_passagem)))

In [15]:
voos

{('FCO', 'LIS'): [('6:19', '8:13', 239),
  ('8:04', '10:59', 136),
  ('9:31', '11:43', 210),
  ('11:07', '13:24', 171),
  ('12:31', '14:02', 234),
  ('14:05', '15:47', 226),
  ('15:07', '17:21', 129),
  ('16:35', '18:56', 144),
  ('18:25', '20:34', 205),
  ('20:05', '21:44', 172)],
 ('LIS', 'FCO'): [('6:11', '8:31', 249),
  ('7:39', '10:24', 219),
  ('9:15', '12:03', 99),
  ('11:08', '13:07', 175),
  ('12:18', '14:56', 172),
  ('13:37', '15:08', 250),
  ('15:03', '16:42', 135),
  ('16:51', '19:09', 147),
  ('18:12', '20:17', 242),
  ('20:05', '22:06', 261)],
 ('FCO', 'MAD'): [('6:03', '8:43', 219),
  ('7:50', '10:08', 164),
  ('9:11', '10:42', 172),
  ('10:33', '13:11', 132),
  ('12:08', '14:47', 231),
  ('14:19', '17:09', 190),
  ('15:04', '17:23', 189),
  ('17:06', '20:00', 95),
  ('18:33', '20:22', 143),
  ('19:32', '21:25', 160)],
 ('MAD', 'FCO'): [('6:05', '8:32', 174),
  ('8:25', '10:34', 157),
  ('9:42', '11:32', 169),
  ('11:01', '12:39', 260),
  ('12:44', '14:17', 134),
  ('14

* Exemplo de dados de voos de Lisboa para Roma:

In [ ]:
voos[(('LIS', 'FCO'))]

[('6:11', '8:31', 249),
 ('7:39', '10:24', 219),
 ('9:15', '12:03', 99),
 ('11:08', '13:07', 175),
 ('12:18', '14:56', 172),
 ('13:37', '15:08', 250),
 ('15:03', '16:42', 135),
 ('16:51', '19:09', 147),
 ('18:12', '20:17', 242),
 ('20:05', '22:06', 261)]

* Agora os voos de volta (Roma para Lisboa):

In [ ]:
voos[('FCO', 'LIS')]

[('6:19', '8:13', 239),
 ('8:04', '10:59', 136),
 ('9:31', '11:43', 210),
 ('11:07', '13:24', 171),
 ('12:31', '14:02', 234),
 ('14:05', '15:47', 226),
 ('15:07', '17:21', 129),
 ('16:35', '18:56', 144),
 ('18:25', '20:34', 205),
 ('20:05', '21:44', 172)]

* Para mapear as respostas, vamos usar uma lista com os índices dos voos escolhidos para cada cidade (ida e volta), conforme ordem especificada na lista `pessoas`. No exemplo abaixo, escolheríamos o voo de índice 1 para ida (Lisboa -> Roma) e o voo de índice 4 para a volta (Roma -> Lisboa) e assim sucessivamente para as outras cidades.
> `[1,4, 3,2, 7,3, 6,3, 2,4, 5,3]`

In [ ]:
pessoas

[('Lisbon', 'LIS'),
 ('Madrid', 'MAD'),
 ('Paris', 'CDG'),
 ('Dublin', 'DUB'),
 ('Brussels', 'BRU'),
 ('London', 'LHR')]

* Precisamos agora de um função para imprimir esses voos de acordo com o vetor supracitado, que chamaremos de `calendario`:

In [16]:
def imprime_calendario(calendario):
    voo_id = -1
    preco_total = 0
    '''calendário possui pares sequenciais dos índices de voos de ida e volta, 
    por isso precisamos acessar dois índices (ida e volta) a cada interação, para cada cidade.
    Dessa forma, precisamos percorrer len(calendario)//2 vezes a lista
    '''
    for i in range (len(calendario)//2):
        nome = pessoas[i][0]
        origem = pessoas[i][1]
        voo_id += 1
        voo_ida = voos[origem, destino][calendario[voo_id]]# índices "pares" são de ida
        voo_id += 1
        preco_total += voo_ida[2] # 2 é o último item da tupla - preço
        voo_volta = voos[destino, origem][calendario[voo_id]]# índices "pares" são de ida
        preco_total += voo_volta[2] # 2 é o último item da tupla - preço
        print(f'{nome:<{10}}{origem:<{10}} {voo_ida[0]:<{5}}-{voo_ida[1]:<{5}} U${voo_ida[2]:<{3}} '+
        f'{voo_volta[0]:<{5}}-{voo_volta[1]:<{5}} U${voo_volta[2]:<{3}}')
    print(f'Preço total: U${preco_total:<{3}}')

* Vamos testar a função:

In [ ]:
imprime_calendario([1,4, 3,2, 7,3, 6,3, 2,4, 5,3])

Lisbon    LIS        7:39 -10:24 U$219 12:31-14:02 U$234
Madrid    MAD        11:01-12:39 U$260 9:11 -10:42 U$172
Paris     CDG        17:07-20:04 U$291 11:08-14:38 U$262
Dublin    DUB        15:27-17:18 U$151 10:33-12:03 U$74 
Brussels  BRU        9:08 -12:12 U$364 12:20-16:34 U$500
London    LHR        13:40-15:38 U$137 10:32-13:16 U$139
Preço total: U$2803


* Agora vamos criar uma função que vai receber como parâmetro os horários dos vôos no formato fornecido (H:M) para minutos, a fim de que possamos calcular o tempo de espera. Primeiro vamos ver como calcularemos esse tempo de espera, com base no exemplo acima:

In [ ]:
#primeiro a chegar: 10:24, último a chegar: 20:04, tempo de espera:580 minutos
10*60 + 24, 20*60 + 4, (20*60 + 4) - (10*60 + 24)

(624, 1204, 580)

In [ ]:
print(f'Maior tempo de espera no voo de ida é {580//60}h e {580%60}min')

Maior tempo de espera no voo de ida é 9h e 40min


* Agora vamos a conversão:

In [ ]:
t = time.strptime('10:24', '%H:%M')

In [ ]:
t

time.struct_time(tm_year=1900, tm_mon=1, tm_mday=1, tm_hour=10, tm_min=24, tm_sec=0, tm_wday=0, tm_yday=1, tm_isdst=-1)

In [ ]:
t[3], t[4], t[3]*60 + t[4]

(10, 24, 624)

In [17]:
def get_minutos(hora):
    t = time.strptime(hora, '%H:%M')
    return t[3]*60 + t[4]

In [ ]:
get_minutos('10:24')

624

### Função de avaliação (fitness function)

A função avaliativa deve calcular a espera total do grupo, tanto na ida quanto na volta (em minutos). Além disso, devemos calcular o preço total das passagens gasto por cada pessoa do grupo. Ao final, retornaremos a soma do preço total com a espera total, já que queremos otimizar essas duas medidas.

In [18]:
def funcao_avaliacao(calendario):    
    #inicializo com o pior valor possível para que possamos otimizá-lo:
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = get_minutos('23:59')

    voo_id = -1
    for i in range(len(calendario) // 2):
        origem = pessoas[i][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]

        preco_total += voo_ida[2]
        preco_total += voo_volta[2]

        if ultima_chegada < get_minutos(voo_ida[1]):
            ultima_chegada = get_minutos(voo_ida[1])
        if primeira_partida > get_minutos(voo_volta[0]):
            primeira_partida = get_minutos(voo_volta[0])

    espera_total = 0
    voo_id = -1
    for i in range(len(calendario) // 2):
        origem = pessoas[i][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]

        espera_total += ultima_chegada - get_minutos(voo_ida[1])
        espera_total += get_minutos(voo_volta[0]) - primeira_partida

    #o retorno é de acordo com o nosso objetivo: minimizar a espera e o preço.
    return espera_total + preco_total

* Vamos agora testar nossa função avaliativa:

In [ ]:
funcao_avaliacao([1,4, 3,1, 8,3, 6,3, 2,4, 5,3])

6096

In [ ]:
imprime_calendario([1,4, 3,1, 8,3, 6,3, 2,4, 5,3])

Lisbon    LIS        7:39 -10:24 U$219 12:31-14:02 U$234
Madrid    MAD        11:01-12:39 U$260 7:50 -10:08 U$164
Paris     CDG        18:23-21:35 U$134 11:08-14:38 U$262
Dublin    DUB        15:27-17:18 U$151 10:33-12:03 U$74 
Brussels  BRU        9:08 -12:12 U$364 12:20-16:34 U$500
London    LHR        13:40-15:38 U$137 10:32-13:16 U$139
Preço total: U$2638


Agora precisamos definir o domínio, isto é, os valores mínimos e máximos que o algoritmo genético poderá escolher para definir os vôos escolhidos.

Nesse caso, existem 10 voos possívies (0-9) para cada trecho (ida e volta). Assim teremos uma lista de 12 opções de 0-9.

In [19]:
dominio = [(0, 9)] * (len(pessoas) * 2)
dominio, len(dominio)

([(0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9),
  (0, 9)],
 12)

#### Operador de Mutação

In [20]:
def mutacao(dominio, passo, calendario):
  gene = random.randint(0, len(dominio) - 1)
  print(gene)

* Nesso caso abaixo o gene sorteado para realizar a mutação foi o sétimo (índice 6).

In [ ]:
mutacao(dominio, 1, [1,4, 3,1, 8,3, 6,3, 2,4, 5,3])

6


* Nesso caso abaixo o gene sorteado para realizar a mutação foi o último (índice 11).

In [ ]:
mutacao(dominio, 1, [1,4, 3,1, 8,3, 6,3, 2,4, 5,3])

11


* Podemos ainda definir a probebilidade de que uma mutação ocorra, com base em um valor fixo de probabilidade (parâmetro `probabilidade` da função abaixo) e um valor sorteado entre 0-1 (usamos a fundação `random.random()`). No caso, se estabelecermos que o valor fixo será 0.2 e o valor sorteado for maior que o valor fixo, então não ocorrerá mutação. A mutação ocorrerá, caso contrário.

* A mudatação será basicamente um decremento de 1, caso o valor do gene seja diferente de 0 e um incremento de 1, caso o valor do gene seja 0.

In [21]:
def mutacao(dominio, passo, calendario, probabilidade):
  gene = random.randint(0, len(dominio) - 1)
  #print(gene)
  mutante = calendario
  if random.random() < probabilidade: # 0.02, 0.05
    if calendario[gene] != dominio[gene][0]:
      mutante = calendario[0:gene] + [calendario[gene] - passo] + calendario[gene + 1:]
      #print(mutante)
    else:
      if calendario[gene] != dominio[gene][1]:
        mutante = calendario[0:gene] + [calendario[gene] + passo] + calendario[gene + 1:]
  return mutante

#### Crossover

Escolheremos o meio da lista como ponto de corte para o crossover. Assim combinaremos a primeira metade de um indíviduo com a segunda metade de outro indivíduo.

In [22]:
def crossover(dominio, individuo1, individuo2):
  gene = random.randint(1, len(dominio) - 2)#ponto de corte - não posso escolher nem a primeira nem a última posição
  #print(gene)
  return individuo1[0:gene] + individuo2[gene:]

In [ ]:
s1 = [1,4, 3,2, 7,3, 6,3, 2,4, 5,3]
s2 = [0,1, 2,5, 8,9, 2,3, 5,1, 0,6]

In [ ]:
crossover(dominio, s1, s2)

9


[1, 4, 3, 2, 7, 3, 6, 3, 2, 1, 0, 6]

### Construindo o fluxo do algoritmo genético

Iremos agora criar o função que processará o fluxo completo do algoritmo genético. Para tanto iremos utilizar os seguintes parâmetros:
* `domínio`: que especifica os limites inferiores e superiores que podem ser escolhidos das opções de voos;
* `funcao_avaliacao`: que retorno o quão apto é um dado indivíduo da população;
* `tamanho_populacao`: aponta o quantidade de indivíduos que farão parte da população que será otimizada pelo algoritmo genético. Observe que neste algoritmo a população inicial será criada aleatoriamente;
* `passo`: é o mesmo parâmetro utilizado na função de mutação. Serve para controlar a magnitude da mutação, isto é, o tamanho do incremento/decremento de um dado gene que sofrerá mutação;
* `elitismo`: aponta o percentual da população que será selecionada para realizar crossover;
* `numero_geracoes`: utilizado como critério de parada. Cada processo de seleção, crossover e mutação de uma dada população é chamado de geração, isto é, uma interação do algorimto genético. Ao chegar no número máximo de gerações passado neste parâmetro, o algoritmo pára.
* `probabilidade_de_mutacao`: é a probibilidade de um de cada indíviduo de uma geração, eleito (após passar pela seleção), sofrer mutação.

Esta função retornará o indivíduo mais apto da última geração avaliada. 

Uma breve explicação de algumas variáveis utilizadas no algoritmo:

* `individuo`: gera-se 12 números aleatórios e os coloca em uma lista chamada `individuo`. Em outras palavras: [random.randint(0, 9) for i in range(12)], entretanto, usaremos os limites inferiores e superiores (0-9) com base na variáveis domínio. Exemplo de um indivíduo: `[3, 7, 9, 9, 5, 9, 6, 5, 2, 8, 2, 9]`;
* `populacao`: a cada interação no primeiro `for`, nós adicionamos o individuo na lista de individuos - `populacao`;
* `numero_elitismo`: aqui nós fazemos a contagem de quantos indivíduos serão selecionados para realizar adaptação (crossover/mutação). Este é um cálculo simples com base na variável elitimso. Por exemplo, o elitismo for de 20% e o tamanho da população for 100, então `numero_elitismo = 0.20*100 = 20`;
* `custos`: para cada indivíduo da população, eu calculo o custo dele com base na função de avaliação. Assim, `custo` será uma lista contendo de tuplas (custo_individuo, individuo);

In [25]:
def algoritmo_genetico(dominio, funcao_avaliacao, tamanho_populacao = 100, passo = 1,
                       elitismo = 0.2, numero_geracoes = 500, probabilidade_mutacao = 0.05):
  populacao = []
  for i in range(tamanho_populacao):
    individuo = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
    populacao.append(individuo)
  #print(populacao)
  #print(len(populacao))  
  numero_elitismo = int(elitismo * tamanho_populacao)
  #print(numero_elitismo)
  #print('Tamanho da população: ', len(populacao))

  for i in range(numero_geracoes):
    custos = [(funcao_avaliacao(individuo), individuo) for individuo in populacao]
    #print(custos)
    custos.sort()
    #print(custos)
    individuos_ordenados = [individuo for (custo, individuo) in custos]
    #print(individuos_ordenados)
    populacao = individuos_ordenados[0:numero_elitismo] # 0:2
    while len(populacao) < tamanho_populacao:
      i1 = random.randint(0, numero_elitismo)
      i2 = random.randint(0, numero_elitismo)
      #print(i1, i2, individuos_ordenados[i1], individuos_ordenados[i2])
      novo_individuo = crossover(dominio, individuos_ordenados[i1], individuos_ordenados[i2])
      #print(novo_individuo)
      mutacao_novo_individuo = mutacao(dominio, passo, novo_individuo, probabilidade_mutacao)
      #print(novo_individuo)
      #print(mutacao_novo_individuo)
      #print()
      populacao.append(mutacao_novo_individuo)

    #print('Tamanho da população: ', len(populacao))
  #print(custos)
  return custos[0][1]

* Agora vamos testar nossa função:

In [26]:
solucao = algoritmo_genetico(dominio, funcao_avaliacao, numero_geracoes = 100, tamanho_populacao=100, 
                             elitismo = 0.2, probabilidade_mutacao = 0.05)

In [27]:
solucao

[7, 3, 6, 3, 6, 4, 6, 3, 5, 3, 7, 3]

In [28]:
funcao_avaliacao(solucao)

2742

In [29]:
imprime_calendario(solucao)

Lisbon    LIS        16:51-19:09 U$147 11:07-13:24 U$171
Madrid    MAD        15:58-18:40 U$173 10:33-13:11 U$132
Paris     CDG        15:34-18:11 U$326 12:37-15:05 U$170
Dublin    DUB        15:27-17:18 U$151 10:33-12:03 U$74 
Brussels  BRU        13:54-18:02 U$294 10:51-14:16 U$256
London    LHR        17:08-19:08 U$262 10:32-13:16 U$139
Preço total: U$2295
